# Nepal Richter Earthquake Damage

## Problem Definition: Which proposed hotel buildings are safe enough to withstand earthquakes?

Team            : 3
Tutorial Group  : B125 (Tuesday) </br>
Lecturer        : Dr Smitha and Ong Chin Ann</br>
Team members and Contribution:
> 1. Jessica Daniella : Extreme Gradient Boosting, K-Nearest Neighbors, Random Forest
> 2. Muhammad Aditya  : Logistic Regression, Decision Tree, Testing

================================ PT. 1 ================================

# DATA EXTRACTION AND DATA PREPARATION 
=====================================================================

#### IMPORT LIBRARY

In [129]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
sb.set()

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score

from sklearn.model_selection import RandomizedSearchCV
from xgboost import plot_importance
from warnings import simplefilter

#### IMPORT DATASETS

In [11]:

train_data = pd.read_csv("./train_values.csv")
train_labels = pd.read_csv("./train_labels.csv")
test_labels = pd.read_csv("./test_values.csv")

#### CHECK DATASETS

In [12]:
print('Data Dimension: ', train_data.shape)

Data Dimension:  (260601, 39)


In [13]:
train_data.columns

Index(['building_id', 'geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_i

In [14]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   building_id                             260601 non-null  int64 
 1   geo_level_1_id                          260601 non-null  int64 
 2   geo_level_2_id                          260601 non-null  int64 
 3   geo_level_3_id                          260601 non-null  int64 
 4   count_floors_pre_eq                     260601 non-null  int64 
 5   age                                     260601 non-null  int64 
 6   area_percentage                         260601 non-null  int64 
 7   height_percentage                       260601 non-null  int64 
 8   land_surface_condition                  260601 non-null  object
 9   foundation_type                         260601 non-null  object
 10  roof_type                               260601 non-null 

In [15]:
train_data.isna().sum()

building_id                               0
geo_level_1_id                            0
geo_level_2_id                            0
geo_level_3_id                            0
count_floors_pre_eq                       0
age                                       0
area_percentage                           0
height_percentage                         0
land_surface_condition                    0
foundation_type                           0
roof_type                                 0
ground_floor_type                         0
other_floor_type                          0
position                                  0
plan_configuration                        0
has_superstructure_adobe_mud              0
has_superstructure_mud_mortar_stone       0
has_superstructure_stone_flag             0
has_superstructure_cement_mortar_stone    0
has_superstructure_mud_mortar_brick       0
has_superstructure_cement_mortar_brick    0
has_superstructure_timber                 0
has_superstructure_bamboo       

In [16]:
train_labels.head()

,building_id,damage_grade
0,802906,3
1,28830,2
2,94947,3
3,590882,2
4,201944,3


In [17]:
train_labels['building_id'].equals(train_data['building_id'])

True

#### Data seems to be OK and no NULL values. Now we merge train_data with train_labels

In [18]:
building_damage = train_data.merge(train_labels, how = 'inner', on = 'building_id')
building_damage.shape

(260601, 40)

In [19]:
building_damage['damage_grade'] = building_damage['damage_grade'].astype('category')
building_damage.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,3
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,2
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,3
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,2
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,3


In [20]:
building_damage[building_damage['age'] < 950]['age'].count()

259211

In [21]:
building_damage.loc[(building_damage['count_families'] == 1) & (building_damage['has_secondary_use_hotel'] == 1)]

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
88,931198,4,1202,5735,1,5,5,3,t,u,...,1,0,0,0,0,0,0,0,0,2
120,824847,3,1387,8014,5,60,3,10,t,r,...,1,0,0,0,0,0,0,0,0,2
155,294469,20,190,9386,2,10,15,6,t,r,...,1,0,0,0,0,0,0,0,0,2
166,670184,24,628,6963,3,15,28,7,t,r,...,1,0,0,0,0,0,0,0,0,2
228,168711,17,289,3449,2,75,6,5,t,r,...,1,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260431,880411,4,514,4669,2,10,16,7,t,r,...,1,0,0,0,0,0,0,0,0,2
260469,1038559,3,1387,9059,5,105,9,11,t,r,...,1,0,0,0,0,0,0,0,0,2
260480,359059,11,883,2822,5,80,11,10,t,r,...,1,0,0,0,0,0,0,0,0,2
260526,220849,17,1005,1989,2,5,11,5,t,r,...,1,0,0,0,0,0,0,0,0,3


#### DATA CLEANING (Remove Outliers)

##### Z-Score

In [22]:
from scipy import stats
z = np.abs(stats.zscore(building_damage.dtypes == np.int64))
print(z)

building_id                               0.538816
geo_level_1_id                            0.538816
geo_level_2_id                            0.538816
geo_level_3_id                            0.538816
count_floors_pre_eq                       0.538816
age                                       0.538816
area_percentage                           0.538816
height_percentage                         0.538816
land_surface_condition                    1.855921
foundation_type                           1.855921
roof_type                                 1.855921
ground_floor_type                         1.855921
other_floor_type                          1.855921
position                                  1.855921
plan_configuration                        1.855921
has_superstructure_adobe_mud              0.538816
has_superstructure_mud_mortar_stone       0.538816
has_superstructure_stone_flag             0.538816
has_superstructure_cement_mortar_stone    0.538816
has_superstructure_mud_mortar_b

In [23]:
building_damage_num_train = building_damage.select_dtypes(include=["number"])
building_damage_cat_train = building_damage.select_dtypes(exclude=["number"])

##### Remove outliers based on their Z-Score

In [24]:
idx = np.all(stats.zscore(building_damage_num_train) < 3, axis=1)
#Outliers Clearance
Q1 = building_damage_num_train.quantile(0.02)
Q3 = building_damage_num_train.quantile(0.98)
IQR = Q3 - Q1
idx = ~((building_damage_num_train < (Q1 - 1.5 * IQR)) | (building_damage_num_train > (Q3 + 1.5 * IQR))).any(axis=1)
train_cleaned = pd.concat([building_damage_num_train.loc[idx], building_damage_cat_train.loc[idx]], axis=1)

In [25]:
train_cleaned

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,...,has_secondary_use_other,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,legal_ownership_status,damage_grade
0,802906,6,487,12198,2,30,6,5,1,1,...,0,t,r,n,f,q,t,d,v,3
1,28830,8,900,2812,2,10,8,7,0,1,...,0,o,r,n,x,q,s,d,v,2
2,94947,21,363,8973,2,10,5,5,0,1,...,0,t,r,n,f,x,t,d,v,3
3,590882,22,418,10694,2,10,6,5,0,1,...,0,t,r,n,f,x,s,d,v,2
4,201944,11,131,1488,3,30,8,9,1,0,...,0,t,r,n,f,x,s,d,v,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,0,1,...,0,n,r,n,f,j,s,q,v,2
260597,669485,17,715,2060,2,0,6,5,0,1,...,0,t,r,n,f,q,s,d,v,3
260598,602512,17,51,8163,3,55,6,7,0,1,...,0,t,r,q,f,q,s,d,v,3
260599,151409,26,39,1851,2,10,14,6,0,0,...,0,t,r,x,v,s,j,d,v,2


In [26]:
train_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243233 entries, 0 to 260600
Data columns (total 40 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   building_id                             243233 non-null  int64   
 1   geo_level_1_id                          243233 non-null  int64   
 2   geo_level_2_id                          243233 non-null  int64   
 3   geo_level_3_id                          243233 non-null  int64   
 4   count_floors_pre_eq                     243233 non-null  int64   
 5   age                                     243233 non-null  int64   
 6   area_percentage                         243233 non-null  int64   
 7   height_percentage                       243233 non-null  int64   
 8   has_superstructure_adobe_mud            243233 non-null  int64   
 9   has_superstructure_mud_mortar_stone     243233 non-null  int64   
 10  has_superstructure_stone_flag   

#### Convert OBJECT data to CATEGORICAL data

In [27]:
num_data=['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq','area_percentage', 'age','height_percentage','count_families']
cat_data=['land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position','legal_ownership_status',
       'plan_configuration'] 
bin_data=['has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
         'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other']

##### Reset INDEX as Building_ID

In [28]:
#Categoricaal Features
cat_feats = ['land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position','legal_ownership_status',
       'plan_configuration']

In [29]:
train_final = pd.get_dummies(building_damage,columns=cat_feats,drop_first=True)
test_final = pd.get_dummies(test_labels,columns=cat_feats,drop_first=True)

In [30]:
# train_final.drop(columns = "has_secondary_use", inplace = True)
train_final.drop(columns = "has_secondary_use_agriculture", inplace = True)

In [31]:
train_final.set_index('building_id', inplace = True)

In [32]:
train_final.describe()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,legal_ownership_status_w,plan_configuration_c,plan_configuration_d,plan_configuration_f,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u
count,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,...,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000
mean,13.900353,701.074685,6257.876148,2.129723,26.535029,8.018051,5.434365,0.088645,0.761935,0.034332,...,0.010272,0.001247,0.959597,0.000084,0.000177,0.000146,0.000610,0.021842,0.001328,0.014002
std,8.033617,412.710734,3646.369645,0.727665,73.565937,4.392231,1.918418,0.284231,0.425900,0.182081,...,0.100831,0.035293,0.196902,0.009188,0.013285,0.012075,0.024693,0.146167,0.036413,0.117500
min,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.000000,350.000000,3073.000000,2.000000,10.000000,5.000000,4.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,12.000000,702.000000,6270.000000,2.000000,15.000000,7.000000,5.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,21.000000,1050.000000,9412.000000,2.000000,30.000000,9.000000,6.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30.000000,1427.000000,12567.000000,9.000000,995.000000,100.000000,32.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
